In [2]:
import sys
import pandas as pd
import math
import operator
trainingsetpath = sys.argv[1]
testsetpath = sys.argv[2]

In [3]:
df = pd.read_csv("data.csv",sep=';')

positiveentropydict = {}
#array that stored columns that doesn't help classifier
deletedcolumnarray = []

correct = 0
incorrect = 0
#create a dictionary which stores the count of different attributes when G = +
positivedict = {}
negativedict = {}
#array store the column where all of the values are same in positive data
positivesamearray = {}
positivecolumncount = 0
#array store the column where all of the values are same in negative data
negativesamearray = {}
total_row = len(df)
maxentropy = {}
bestratioarray = []
smallesterror = 1
#number of mispredict for each model
numerror = 0
#threshold for deciding top 20%
threshold = 0
bestpositivedict = {}
bestnegativedict = {}
#increment row index for validation data(5-fold cross validation)
row_index_increment = (int)(total_row/5)



#try 10 sets of combinations of different attributes and compare their mean value of 5-fold cross validation
for sets in range(10):
    if(sets != 0):
        del positiveentropydict[maxentropy]
        del positivedict[maxentropy]
        del negativedict[maxentropy]
        df = df.drop(columns=maxentropy)
    positiveratioarray = []
    numerror = 0
    #loop through to make sure every data has been tested as validation data
    #5-fold cross validation
    for fold in range (5):
        
        #situation when row number is smaller than 5
        if(len(df) < 5):
            if(fold < len(df)):
                starting_index = fold
                end_index = fold
                validation_data = df.loc[fold:fold]
            else:
                starting_index = len(df)-1
                end_index = len(df) - 1
                validation_data = df.loc[len(df)-1]
        elif(fold < 4):
            starting_index = fold*row_index_increment
            end_index = fold*row_index_increment+row_index_increment-1
            validation_data = df.loc[fold*row_index_increment:fold*row_index_increment+row_index_increment-1]
        elif(fold == 4):
            starting_index = fold*row_index_increment
            end_index = len(df)-1
            validation_data = df.loc[fold*row_index_increment:len(df)-1]
        training_data = df
        for i in range(starting_index,end_index+1):
            training_data = training_data.drop(df.index[i])
        #validation_data.reset_index()
        #training_data.reset_index()
        positivearray = training_data.loc[df['G'] == '+']
        positiverownumber = len(positivearray)
        positivearray = positivearray.reset_index()
        negativearray = training_data.loc[df['G'] == '-']
        negativerownumber = len(negativearray)
        negativearray = negativearray.reset_index()
        #positive grade array
        for col in positivearray.columns:
            columnarray = positivearray[col].values
            #create a dictionary that store results of single one attribute
            arraydict = {}

            #add a element to count the total number
            arraydict['totalcount'] = 0
            for element in columnarray:
                arraydict['totalcount'] = arraydict['totalcount'] + 1
                #if the element in columnarray is not in the dictionary
                if element not in arraydict:
                    arraydict[element] = 1
                else:
                    arraydict[element] = arraydict[element] + 1

            #calculate maximum frequency for current column
            totalcount = 0
            entropy = 0
            if (col != "index"):
                for key,count in arraydict.items():
                    if(key == "totalcount"):
                        totalcount = count
                    elif(count == totalcount):
                        positivesamearray[col] = key
                    else:
                        entropy -= count/totalcount*math.log2(count/totalcount)

                positiveentropydict[col] = entropy

            if(col != "index"):
                #add array dict into positive dictionary
                positivedict[col] = arraydict;
                positivecolumncount += 1;


        #negative grade array    
        negativecolumncount = 0
        for col in negativearray.columns:
            columnarray = negativearray[col].values
            #create a dictionary that store results of single one attribute
            arraydict = {}
            arraydict['totalcount'] = 0

            for element in columnarray:
                arraydict['totalcount'] = arraydict['totalcount'] + 1
                #if the element in columnarray is not in the dictionary
                if element not in arraydict:
                    arraydict[element] = 1
                else:
                    arraydict[element] = arraydict[element] + 1

            #calculate maximum frequency for current column
            totalcount = 0
            if (col != "index"):
                for key,count in arraydict.items():
                    if(key == "totalcount"):
                        totalcount = count
                    elif(count == totalcount):
                        negativesamearray[col] = key


            #add array dict into positive dictionary
            if(col != "index"):
                negativedict[col] = arraydict;
                negativecolumncount += 1


        #I don't need index count, so I delete them
        #del positivedict[0]
        #del negativedict[0]
    

        #find out the column that does not make contribution to classify
        for key,value in positivesamearray.items():
            if key in negativesamearray:
                if(positivesamearray[key] == negativesamearray[key]):
                    deletedcolumnarray.append(key)

        for key in deletedcolumnarray:
            if key in positivedict:
                del positivedict[key]
                del negativedict[key]
                del positiveentropydict[key]
    
        maxentropy = max(positiveentropydict.items(), key=operator.itemgetter(1))[0]
        
        #predict validation data
        validation_data.reset_index()


        #print(df)
        #row = next(df.iterrows())[1]
        #check for validation result
        totaltest = 0
        for validateindex in range(0,len(validation_data)):
            positivelikelihood = 0
            negativelikelihood = 0
            for key,value in positivedict.items():
                if(key != "G"):
                    
                    singledata = validation_data[key].iloc[validateindex]
                    #get the possibility
                    testcount = 0
                    if singledata not in positivedict[key]:
                        testcount = 0
                    else:
                        testcount = positivedict[key][singledata]
                    if singledata not in negativedict[key]:
                        ntestcount = 0
                    else:
                        ntestcount = negativedict[key][singledata]
                    totalcount = positivedict[key]["totalcount"]
                    ntotalcount = negativedict[key]["totalcount"]
                    #laplace correction
                    positivelikelihood += math.log2((testcount+1)/(totalcount+len(df.columns)))
                    negativelikelihood += math.log2((ntestcount+1)/(ntotalcount+len(df.columns)))
            positivelikelihood += math.log2(totalcount/len(df))
            negativelikelihood += math.log2(ntotalcount/len(df))
            positiverate = positivelikelihood/(positivelikelihood+negativelikelihood)
            
            if(positivelikelihood > negativelikelihood):
                positiveratioarray.append(positiverate)
                if(validation_data['G'].iloc[validateindex] == '-'):
                    #print("error\n")
                    numerror += 1
            else:
                if(validation_data['G'].iloc[validateindex] == '+'):
                    #print("error\n")
                    numerror += 1
            #print("positivelikelihood",positivelikelihood)
            #print("negativelikelihood",negativelikelihood,'\n')
    errorrate = numerror/len(df)
    if(errorrate < smallesterror):
        print(positivedict)
        smallesterror = errorrate
        bestpositivedict = positivedict
        bestnegativedict = negativedict
        bestratioarray = positiveratioarray
    positiveratioarray = []
bestratioarray.sort(reverse=False)
threshold_index = (int)(len(bestratioarray)/5)
threshold = bestratioarray[threshold_index]

#data prediction
test_data = pd.read_csv("data_test.csv",sep=';')
for index,row in test_data.iterrows():
    positivelikelihood = 0
    negativelikelihood = 0
    for key,value in bestpositivedict.items():
        if(key != "G"):
            singledata = row[key]
            testcount = 0
            ntestcount = 0
            if singledata not in bestpositivedict[key]:
                testcount = 0
            else:
                testcount = bestpositivedict[key][singledata]
            if singledata not in bestnegativedict[key]:
                ntestcount = 0
            else:
                ntestcount = bestnegativedict[key][singledata]
            totalcount = positivedict[key]["totalcount"]
            ntotalcount = negativedict[key]["totalcount"]
            positivelikelihood += math.log2((testcount+1)/(totalcount+len(df.columns)))
            negativelikelihood += math.log2((ntestcount+1)/(ntotalcount+len(df.columns)))
    positivelikelihood += math.log2(totalcount/(totalcount+ntotalcount))
    negativelikelihood += math.log2(ntotalcount/(totalcount+ntotalcount))
    if(positivelikelihood > negativelikelihood):
        positiverate = positivelikelihood/(positivelikelihood+negativelikelihood)
    else:
        positiverate = 1
    if(positiverate <= threshold):
        
        print('+')
        
    else:
        if(row['G'] == '+'):
            correct += 1
        else:
            incorrect += 1
        print('-')     
            #print(validaterow.iloc[0])
            #print('\n')
c = test_data.loc[df['G'] == '-']
print(len(c))
print(incorrect)
        
            
        
           
        
      
    
    

{'sex': {'totalcount': 197, 'F': 126, 'M': 71}, 'age': {'totalcount': 197, 15: 36, 16: 57, 17: 62, 18: 39, 19: 2, 20: 1}, 'address': {'totalcount': 197, 'U': 170, 'R': 27}, 'famsize': {'totalcount': 197, 'GT3': 148, 'LE3': 49}, 'Pstatus': {'totalcount': 197, 'T': 170, 'A': 27}, 'Medu': {'totalcount': 197, 4: 82, 3: 45, 2: 49, 1: 20, 0: 1}, 'Fedu': {'totalcount': 197, 2: 55, 3: 51, 4: 57, 1: 33, 0: 1}, 'Mjob': {'totalcount': 197, 'health': 22, 'other': 73, 'services': 47, 'teacher': 32, 'at_home': 23}, 'Fjob': {'totalcount': 197, 'services': 44, 'other': 111, 'teacher': 21, 'health': 10, 'at_home': 11}, 'reason': {'totalcount': 197, 'home': 60, 'reputation': 56, 'course': 70, 'other': 11}, 'guardian': {'totalcount': 197, 'mother': 150, 'father': 42, 'other': 5}, 'traveltime': {'totalcount': 197, 1: 140, 2: 47, 3: 8, 4: 2}, 'studytime': {'totalcount': 197, 3: 45, 2: 101, 1: 37, 4: 14}, 'failures': {'totalcount': 197, 0: 194, 1: 2, 2: 1}, 'schoolsup': {'totalcount': 197, 'no': 181, 'yes':